In [1]:
import pandas as pd
import re
import urllib.request
from collections import Counter
from insta_scrape import insta_details, find_hashtags, get_full_info, get_posts, get_hashtags
from selenium.webdriver import Chrome, Firefox
import json

### Hashtags to scrape: 
1. travel (Anna)
2. food
3. animals
4. selfie
5. cars
6. fitness
7. babies
8. wedding
9. nature
10. architecture

In [10]:
# scrape info for hashtag 'travel'

travel = get_full_info('travel', 300)

300 300


In [24]:
#cast to dataframe

df_travel = pd.DataFrame(travel)
df_travel.head()

,hashtags,image,image_local_name,post_link,search_hashtag
0,"[#teleport, #photoshoot, #instafashion, #trave...",https://scontent-lga3-1.cdninstagram.com/vp/28...,e66420a7-1fc1-49a7-a649-5c591f1d60b8.jpg,https://www.instagram.com/p/By3HaZNgmnE/,travel
1,"[#lagodibraies, #mountains, #lacs, #travel, #l...",https://scontent-lga3-1.cdninstagram.com/vp/33...,b6316377-299c-4f5a-b17e-c7879dce5220.jpg,https://www.instagram.com/p/By3G3bpi15L/,travel
2,[],https://scontent-lga3-1.cdninstagram.com/vp/52...,5f25e1a8-ad7f-4d8b-b0b4-ca7ff3930720.jpg,https://www.instagram.com/p/By3HdV1F6zI/,travel
3,"[#wanderlust, #mallorca, #balearicislands, #am...",https://scontent-lga3-1.cdninstagram.com/vp/cb...,c0077760-c457-46f0-951a-9a1d8b941030.jpg,https://www.instagram.com/p/By3IE3An3x2/,travel
4,"[#eros, #nikon, #photooftheday, #instalike, #i...",https://scontent-lga3-1.cdninstagram.com/vp/4b...,1e1293dc-7033-4c67-aca2-f00397aa96f2.jpg,https://www.instagram.com/p/By3KDrwnjM6/,travel


In [25]:
#export in a json file

with open('json_files/travel.json', 'w') as fp:
    json.dump(travel, fp)